In [1]:
from llm.factory import LLMInterface
from setting.db import SessionLocal
from llm.embedding import get_text_embedding
from knowledgebase.doc_builder import DocBuilder

# llm_client = LLMInterface("bedrock", "us.deepseek.r1-v1:0")
llm_client = LLMInterface("bedrock", "arn:aws:bedrock:us-east-1:841162690310:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0")
doc_builder = DocBuilder(llm_client, get_text_embedding)

In [ ]:
sections = doc_builder.split_markdown_by_heading("../graph/docs/programming-guides/proto3.md", {
    "doc_version": "1.0",
    "category": "Programming Guides",
})

In [ ]:
from knowledgebase.doc_model import KnowledgeBlock

from sqlalchemy.orm import joinedload

extend_blocks = []
with SessionLocal() as db:
    # Query for complete entities
    blocks = db.query(KnowledgeBlock).options(
        joinedload(KnowledgeBlock.source)
    ).all()
    

    # Now you can access all attributes
    for block in blocks:
        extend_blocks.append({
            'id': block.id,
            'name': block.name,
            'content': block.content,
            'source_content': block.source.content
        })

contexts = doc_builder.agument_block_context(extend_blocks)
contexts


In [ ]:
doc_builder.analyze_concepts(topic="How to update protobuf definition without breaking the compatibility")

In [2]:
from knowledgebase.doc_search import search_knowledge_blocks

search_knowledge_blocks("How to update protobuf file?", context_weight=0.6)

{'concepts': [{'name': 'Tag Reuse Issues',
   'definition': 'Problems that can arise when modifying oneof fields in protobuf definitions. These include: moving fields into/out of oneofs (may lose information), deleting and re-adding oneof fields (may clear current values), and splitting/merging oneofs (similar issues to moving singular fields).',
   'source_ids': ['5fec9a80-b360-4bcd-b7f0-3b3be6b41ce6'],
   'distance': 0.5593309745977644},
  {'name': 'Unknown Fields',
   'definition': "Well-formed protocol buffer data representing fields that a parser doesn't recognize, typically when an old binary parses data from a newer version with new fields. Proto3 preserves unknown fields during parsing and serialization, maintaining backward compatibility.",
   'source_ids': ['7f62597b-e040-4f19-b725-d4fa66dc4476'],
   'distance': 0.5696796382829501},
  {'name': 'Backwards Compatibility with Maps',
   'definition': "Maps in protobuf are equivalent to a specific repeated message structure on the